In [5]:
#import necessari
import torch
#import transformer
import transformer_norm
import os
import utils
import time
from PIL import Image


In [9]:
#### Generazione Imagini trasferimento stile 
# il codice chiederà di inserire il path della cartella con tutte le imagini da trasformare
# inserire il path completo tipo C:\Users\Name\Desktop\Folder 
# oppure inserire solo le sottocartelle della cartella corrente se il file si trova gia in Desktop inserire solo Folder
# se si usa il tranformer con i meccanismi di attenzione alcune imagini verranno skippate
# se si voglono comunque processare usare il codice sottostante a questo

# Pulizia cache GPU
torch.cuda.empty_cache() 
# Inserire path dei pesi della rete addestrata da utilizzare
STYLE_TRANSFORM_PATH = "/DL_Prog/content/kill/coco/transformer_weight.pth"
#STYLE_TRANSFORM_PATH = "/DL_Prog/content/kill/coco/checkpoint_16000.pth"
# Impostare False per non mantenere i colori originali dell'immagine di contenuto (vengono usati i colori dell'immagine di stile)
# Impostare True per mantenere i colori originali dell'immagine di contenuto 
PRESERVE_COLOR = False

def stylize_folder(content_folder, style_transform_path, preserve_color=False):
    """
    Stylizes all images in the content_folder and saves them in new nested folders based on the
    style_transform_path.
    
    Parameters:
    - content_folder: Path to the folder containing content images.
    - style_transform_path: Path to the style transformer model.
    - preserve_color: Boolean indicating whether to preserve the colors of the content images.
    """
    # Device
    device = ("cuda" if torch.cuda.is_available() else "cpu")

    ##############################################################################################################################################################
    # Load Transformer Network SELEZIONARE IL TRANF GIUSTO
    ##########################################################################################################################################################
    # Caricamento dei pesi della rete di trasformazione di stile e trasferimento del modello sul dispositivo scelto.
    
    #net = transformer.TransformerNetworkTanhWithAttention()
    net = transformer_norm.TransformerNetworkTanh()

    net.load_state_dict(torch.load(style_transform_path))
    net = net.to(device)

    # Extract the first and second subfolder names from the style_transform_path
    first_subfolder_name = style_transform_path.split('/')[3]
    second_subfolder_name = '/'.join(style_transform_path.split('/')[4:-1])

    # Create output folder with the extracted subfolder names
    output_folder = os.path.join(content_folder, "output", first_subfolder_name, second_subfolder_name)
    os.makedirs(output_folder, exist_ok=True)

    # Stylize every image in the content folder
    # Recupero di tutte le immagini compatibili (JPG, JPEG, PNG) dalla cartella di contenuti e iterazione su ciascuna. 
    images = [img for img in os.listdir(content_folder) if img.endswith((".jpg", ".jpeg", ".png"))]
    with torch.no_grad():
        for image_name in images:
            try:
                # Free-up unneeded cuda memory
                torch.cuda.empty_cache()

                # Load content image
                content_image_path = os.path.join(content_folder, image_name)
                content_image = utils.load_image(content_image_path)
                content_tensor = utils.itot(content_image).to(device)

                # Generate stylized image
                starttime = time.time()
                generated_tensor = net(content_tensor)
                generated_image = utils.ttoi(generated_tensor.detach())
                if preserve_color:
                    generated_image = utils.transfer_color(content_image, generated_image)
                print(f"Stylized {image_name} in {time.time() - starttime} seconds.")

                # Save stylized image
                output_image_path = os.path.join(output_folder, image_name)
                utils.saveimg(generated_image, output_image_path)
                
            except RuntimeError as e:
                # Catch out-of-memory errors and skip the image
                if 'out of memory' in str(e):
                    
                    print(f"Skipping {image_name} due to memory limitations.")
                    torch.cuda.empty_cache()
                else:
                    raise e

    print(f"All images stylized and saved in {output_folder}")

# Example usage
content_folder = input("Enter the folder path containing the images: ")
stylize_folder(content_folder, STYLE_TRANSFORM_PATH, PRESERVE_COLOR)



C:\Users\vital\AppData\Local\Temp\ipykernel_7732\3102023421.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(style_transform_path))


Stylized 70jq6imm9g57.jpg in 0.2871739864349365 seconds.
Stylized aqua.jpg in 0.07106471061706543 seconds.
Stylized colosseum-rome-dusk-1-1.jpg in 0.025022506713867188 seconds.
Stylized drake.jpg in 0.08107423782348633 seconds.
Stylized gojo.jpg in 0.038034915924072266 seconds.
Stylized guts_angry.jpg in 0.017015695571899414 seconds.
Stylized image.jpg in 0.21919894218444824 seconds.
Stylized image1.jpg in 0.039034366607666016 seconds.
Stylized IMG1.jpg in 0.019015789031982422 seconds.
Stylized jinx.jpg in 0.0790717601776123 seconds.
Stylized P1.jpg in 0.6345763206481934 seconds.
Stylized Pantheon_Rom_1_cropped.jpg in 0.021020174026489258 seconds.
Stylized PXL_20230908_230949088.jpg in 0.9469826221466064 seconds.
Stylized PXL_20230909_130056726.jpg in 0.6342096328735352 seconds.
Stylized PXL_20240412_104913318.jpg in 0.7056410312652588 seconds.
Stylized PXL_20240413_095450371 - Copia.jpg in 0.02202010154724121 seconds.
Stylized PXL_20240413_095450371.jpg in 0.46942567825317383 seconds.

In [19]:
#resized con limiti massimi stimati 
# Clean up GPU cache
torch.cuda.empty_cache()

# Path to the pre-trained style transformer model
STYLE_TRANSFORM_PATH = "/DL3/content/Van/coco1_att/transformer_weight.pth"
#STYLE_TRANSFORM_PATH = "/DL3/content/Van/coco2/checkpoint_16000.pth"
# Set to True to preserve original content image colors, False to use style image colors
PRESERVE_COLOR = True
#set the max_size = to a value of pixel
#limite massimo per 16gb 700*700, se le immagini sono 16:9 limite suggerito 


def resize_image(image_path):
    with Image.open(image_path) as img:
        width, height = img.size

        # Calcolo del rapporto di aspetto
        aspect_ratio = width / height
        if width > height:
            aspect_ratio = width / height
        else:
            aspect_ratio = height / width

        # Definizione del limite massimo in base al rapporto di aspetto
        if aspect_ratio >= 16/9:
            max_size = 900
        else:
            max_size = 700  # Limite predefinito se il rapporto non è 16:9 o 1:1

        # Verifica se il ridimensionamento è necessario
        if max(width, height) > max_size:
            if width > height:
                new_width = max_size
                new_height = int(max_size * height / width)
            else:
                new_height = max_size
                new_width = int(max_size * width / height)
            
            resized_img = img.resize((new_width, new_height), Image.LANCZOS)
            
            # Crea la sottocartella 'resized'
            resized_folder = os.path.join(os.path.dirname(image_path), 'resized')
            os.makedirs(resized_folder, exist_ok=True)
            
            # Salva l'immagine ridimensionata con suffisso "_resized" nella sottocartella 'resized'
            base_name, ext = os.path.splitext(os.path.basename(image_path))
            resized_image_path = os.path.join(resized_folder, f"{base_name}_resized{ext}")
            resized_img.save(resized_image_path)
            print(f"Resized {image_path} to {new_width}x{new_height} and saved as {resized_image_path}.")
            return resized_image_path
        return image_path


def stylize_folder(content_folder, style_transform_path, preserve_color=False):
    """
    Stylizes all images in the content_folder and saves them in new nested folders based on the
    style_transform_path.
    
    Parameters:
    - content_folder: Path to the folder containing content images.
    - style_transform_path: Path to the style transformer model.
    - preserve_color: Boolean indicating whether to preserve the colors of the content images.
    """
    # Device
    device = ("cuda" if torch.cuda.is_available() else "cpu")

    ##############################################################################################################################################################
    # Load Transformer Network SELEZIONARE IL TRANF GIUSTO
    ##############################################################################################################################################################
    # Load the style transformer network weights and transfer the model to the chosen device.
    net = transformer.TransformerNetworkTanhWithAttention()
    net.load_state_dict(torch.load(style_transform_path))
    net = net.to(device)

    # Extract the first and second subfolder names from the style_transform_path
    first_subfolder_name = style_transform_path.split('/')[3]
    second_subfolder_name = '/'.join(style_transform_path.split('/')[4:-1])

    # Create output folder with the extracted subfolder names
    output_folder = os.path.join(content_folder, "output", first_subfolder_name, second_subfolder_name)
    os.makedirs(output_folder, exist_ok=True)

    # Stylize every image in the content folder
    images = [img for img in os.listdir(content_folder) if img.endswith((".jpg", ".jpeg", ".png"))]
    with torch.no_grad():
        for image_name in images:
            content_image_path = os.path.join(content_folder, image_name)
            
            # Resize image if necessary and get the path of the resized image
            resized_image_path = resize_image(content_image_path)

            try:
                # Free-up unneeded cuda memory
                torch.cuda.empty_cache()

                # Load content image
                content_image = utils.load_image(resized_image_path)
                content_tensor = utils.itot(content_image).to(device)

                # Generate stylized image
                starttime = time.time()
                generated_tensor = net(content_tensor)
                generated_image = utils.ttoi(generated_tensor.detach())
                if preserve_color:
                    generated_image = utils.transfer_color(content_image, generated_image)
                print(f"Stylized {image_name} in {time.time() - starttime} seconds.")

                # Save stylized image
                output_image_path = os.path.join(output_folder, image_name)
                utils.saveimg(generated_image, output_image_path)
                
            except RuntimeError as e:
                # Catch out-of-memory errors and print the error
                if 'out of memory' in str(e):
                    print(f"Error processing {image_name}: {e}.")
                    torch.cuda.empty_cache()
                else:
                    raise e

    print(f"All images stylized and saved in {output_folder}")

# Example usage
content_folder = input("Enter the folder path containing the images: ")
stylize_folder(content_folder, STYLE_TRANSFORM_PATH, PRESERVE_COLOR)


C:\Users\vital\AppData\Local\Temp\ipykernel_2016\2269236996.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(style_transform_path))


Stylized 70jq6imm9g57.jpg in 0.37085962295532227 seconds.
Stylized 70jq6imm9g571.jpg in 0.14212942123413086 seconds.
Resized images/input\aqua.jpg to 900x506 and saved as images/input\resized\aqua_resized.jpg.
Stylized aqua.jpg in 0.2210242748260498 seconds.
Stylized colosseum-rome-dusk-1-1.jpg in 0.15013647079467773 seconds.
Resized images/input\drake.jpg to 700x700 and saved as images/input\resized\drake_resized.jpg.
Stylized drake.jpg in 0.25122857093811035 seconds.
Resized images/input\gojo.jpg to 700x700 and saved as images/input\resized\gojo_resized.jpg.
Stylized gojo.jpg in 0.25623345375061035 seconds.
Stylized guts_angry.jpg in 0.08507752418518066 seconds.
Resized images/input\image.jpg to 700x472 and saved as images/input\resized\image_resized.jpg.
Stylized image.jpg in 0.12211179733276367 seconds.
Stylized image1.jpg in 0.1191093921661377 seconds.
Stylized IMG1.jpg in 0.11328339576721191 seconds.
Resized images/input\jinx.jpg to 700x394 and saved as images/input\resized\jinx_